## AdaBoost

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### AdaBoost Classifier from scratch in python

We will implement ensemble AdaBoost Classifier with Decision Stump.

In [4]:
y = np.array(['yes', 'no', 'no', 'yes', 'no', 'yes', 'no', 'yes'])

classes_ = np.unique(y)

classes_[0], classes_[1]

y_encoded = np.where(y == classes_[0], -1, 1)

y_encoded

array([ 1, -1, -1,  1, -1,  1, -1,  1])

In [6]:
n_samples = len(y)

np.ones(n_samples)/n_samples

array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])

In [ ]:
class AdaBoost:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.alphas = []
        self.models = []
        self.epsilon = 1e-8
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        # Convert labels to {-1, +1} format for AdaBoost
        self.classes_ = np.unique(y)
        y_encoded = np.where(y == self.classes_[0], -1, 1)
        
        # Initialize weights uniformly
        w = np.ones(n_samples) / n_samples
        
        for t in range(self.n_estimators):
            # Train weak learner (decision stump), we can use other type of weak learner as well
            model = DecisionTreeClassifier(max_depth=1, random_state=42)
            model.fit(X, y, sample_weight=w)  # we are using weighted gini
            
            # Get predictions and convert to {-1, +1}
            predictions = model.predict(X)
            predictions_encoded = np.where(predictions == self.classes_[0], -1, 1)
            
            # Calculate weighted error
            error = np.sum(w * (predictions_encoded != y_encoded)) / np.sum(w)
            
            # Handle edge cases for error
            if error >= 0.5:
                # If error >= 0.5, weak learner is no better than random
                if len(self.models) == 0:
                    # First iteration, continue with small adjustment
                    error = 0.5 - self.epsilon
                else:
                    # Stop adding more weak learners
                    break
            
            if error <= 0:
                error = self.epsilon
            
            # Calculate alpha (weak learner influence)
            alpha = 0.5 * np.log((1 - error) / error)
            
            # Store model and alpha
            self.models.append(model)
            self.alphas.append(alpha)
            
            # Update sample weights
            w = w * np.exp(-alpha * y_encoded * predictions_encoded)
            w = w / np.sum(w)  # Normalize weights
    
    def predict(self, X):
        if not self.models:
            raise ValueError("Model has not been fitted yet.")
        
        # Initialize predictions
        final_predictions = np.zeros(X.shape[0])
        
        # Combine predictions from all weak learners
        for model, alpha in zip(self.models, self.alphas):
            predictions = model.predict(X)
            predictions_encoded = np.where(predictions == self.classes_[0], -1, 1)
            final_predictions += alpha * predictions_encoded
        
        # Convert back to original label format
        binary_predictions = np.where(final_predictions >= 0, 1, -1)
        return np.where(binary_predictions == -1, self.classes_[0], self.classes_[1])
    
    def predict_proba(self, X):
        """Predict class probabilities"""
        if not self.models:
            raise ValueError("Model has not been fitted yet.")
        
        final_predictions = np.zeros(X.shape[0])
        
        for model, alpha in zip(self.models, self.alphas):
            predictions = model.predict(X)
            predictions_encoded = np.where(predictions == self.classes_[0], -1, 1)
            final_predictions += alpha * predictions_encoded
        
        # Convert to probabilities using sigmoid-like transformation
        prob_class_1 = 1 / (1 + np.exp(-2 * final_predictions))
        prob_class_0 = 1 - prob_class_1
        
        return np.column_stack((prob_class_0, prob_class_1))


### Create synthtetic dataset and measure performance

In [8]:
# Create a synthtic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Create a ensemble Adaboost classifier
adaboost = AdaBoost(n_estimators=50)

# Fitting the classifier
adaboost.fit(X_train, y_train)

# Predict on test dataset
predictions = adaboost.predict(X_test)

# Measure the performance
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f"Accuracy: {accuracy * 100:.4f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 85.3333%
Precision: 0.8194
Recall: 0.8881
F1 Score: 0.8523


### Sklearn in-built AdaBoost Classifier

sklearn adaboost classifier documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

sklearn adaboost regressor documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html

In [9]:
from sklearn.ensemble import AdaBoostClassifier

In [10]:
# define the model. The estimator parameter is optional. By default it will take a decision stump
adaboost_sklearn = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                                      n_estimators=50,
                                      random_state=42)

# fitting the model
adaboost_sklearn.fit(X_train, y_train)

# Predict on test dataset
predictions = adaboost_sklearn.predict(X_test)

# Measure the performance
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f"Accuracy: {accuracy * 100:.4f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 85.3333%
Precision: 0.8194
Recall: 0.8881
F1 Score: 0.8523
